# TrustyAI

## Load data

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

df = pd.read_csv("../../datasets/FICO/heloc_dataset_v1.csv")

In [3]:
import utils

bounds = utils.data_bounds(df)

## Initialise TrustyAI

In [4]:
import trustyai
import os
import site

DEFAULT_DEP_PATH = os.path.join(site.getsitepackages()[0], "trustyai", "dep")

CORE_DEPS = [
    f"../../deps/*",
    f"{DEFAULT_DEP_PATH}/org/slf4j/slf4j-api/1.7.30/slf4j-api-1.7.30.jar",
    f"{DEFAULT_DEP_PATH}/org/apache/commons/commons-lang3/3.12.0/commons-lang3-3.12.0.jar",
    f"{DEFAULT_DEP_PATH}/org/optaplanner/optaplanner-core-impl/8.18.0.Final/"
    f"optaplanner-core-impl-8.18.0.Final.jar",
    f"{DEFAULT_DEP_PATH}/org/apache/commons/commons-math3/3.6.1/commons-math3-3.6.1.jar",
    f"{DEFAULT_DEP_PATH}/org/kie/kie-api/8.18.0.Beta/kie-api-8.18.0.Beta.jar",
    f"{DEFAULT_DEP_PATH}/io/micrometer/micrometer-core/1.8.2/micrometer-core-1.8.2.jar",
]

trustyai.init(path=CORE_DEPS)

## XGBoost

### Load model

In [5]:
from xgboost.sklearn import XGBClassifier

model = XGBClassifier()

In [6]:
model.load_model("../../models/xgboost.json")

### Selecting inputs

Select the input $X$ with a probability of negative outcome closest to $75\%$.

In [7]:
X = utils.get_negative_closest(model, 0.75)

In [8]:
X.to_dict()

{'ExternalRiskEstimate': 63,
 'MSinceOldestTradeOpen': 309,
 'MSinceMostRecentTradeOpen': 36,
 'AverageMInFile': 112,
 'NumSatisfactoryTrades': 17,
 'NumTrades60Ever2DerogPubRec': 2,
 'NumTrades90Ever2DerogPubRec': 1,
 'PercentTradesNeverDelq': 68,
 'MSinceMostRecentDelq': 7,
 'MaxDelq2PublicRecLast12M': 2,
 'MaxDelqEver': 4,
 'NumTotalTrades': 19,
 'NumTradesOpeninLast12M': 0,
 'PercentInstallTrades': 16,
 'MSinceMostRecentInqexcl7days': 0,
 'NumInqLast6M': 1,
 'NumInqLast6Mexcl7days': 1,
 'NetFractionRevolvingBurden': 54,
 'NetFractionInstallBurden': -8,
 'NumRevolvingTradesWBalance': 5,
 'NumInstallTradesWBalance': 1,
 'NumBank2NatlTradesWHighUtilization': 2,
 'PercentTradesWBalance': 67}

Check the model's output for $X$

In [9]:
model.predict_proba(X.to_numpy().reshape(1, -1))

array([[0.75024784, 0.24975218]], dtype=float32)

TrustyAI wrapper for the XGBoost model

In [10]:
from org.kie.kogito.explainability.model import PredictionInput, PredictionOutput
from trustyai.model import output
import numpy as np

TARGET = "RiskPerformance"

def predict(_model):
    def fun(inputs):
        values = [_feature.value.as_number() for _feature in inputs[0].features]
        result = _model.predict_proba(np.array([values]))
        bad_prob, good_prob = result[0]
        if bad_prob > good_prob:
            _prediction = (0, bad_prob)
        else:
            _prediction = (1, good_prob)
        _output = output(name=TARGET, dtype="number", value=_prediction[0], score=_prediction[1])
        return [PredictionOutput([_output])]
    return fun

In [11]:
from trustyai.model import Model

provider = Model(predict(model))

Build input features with a fixed search bound of $[-20, 1000]$

In [12]:
from trustyai.model import feature

input_feature = []
input_dict = X.to_dict()
for name in input_dict:
    input_feature.append(
        feature(name=name, value=input_dict[name], dtype="number",
                domain=(-20, 1000))
    )

In [13]:
predict(model)([PredictionInput(input_feature)])[0].outputs.get(0).toString()

'Output{value=0, type=number, score=0.7502478361129761, name='RiskPerformance'}'

Define the counterfactual goal as the label `1` ("Good" assessment)

In [14]:
goal = [output(name=TARGET, dtype="number", value=1)]

In [15]:
goal[0].toString()

'Output{value=1, type=number, score=1.0, name='RiskPerformance'}'

In [16]:
from trustyai.model import counterfactual_prediction

prediction = counterfactual_prediction(
    input_features=input_feature,
    outputs=goal)

Create the explainer with a maximum of $100,000$ iterations.

In [19]:
from trustyai.explainers import CounterfactualExplainer

explainer = CounterfactualExplainer(steps=100_000)

Run the explainer

In [20]:
%%time

explanation = explainer.explain(prediction, provider)

CPU times: user 10min 38s, sys: 21min 54s, total: 32min 32s
Wall time: 3min 35s


Extract the features from the counterfactual explanation

In [21]:
expl_features = [e.asFeature() for e in explanation.entities]

In [22]:
cf_p = predict(model)([PredictionInput(expl_features)])[0].outputs.get(0)
cf_p = {
    "GoalValue": [cf_p.getValue().asNumber()],
    "GoalName": [str(cf_p.getName())],
    "GoalScore": [cf_p.getScore()],
}
print(cf_p)

{'GoalValue': [1.0], 'GoalName': ['RiskPerformance'], 'GoalScore': [0.5726234912872314]}


In [23]:
def show_changes(explanation, original):
    entities = explanation.entities
    N = len(original)
    for i in range(N):
        name = original[i].name
        original_value = original[i].value.as_number()
        new_value = entities[i].as_feature().value.as_number()
        if original_value != new_value:
            print(f"Feature '{name}': {original_value} -> {new_value}")


show_changes(explanation, input_feature)

Feature 'MaxDelq2PublicRecLast12M': 2.0 -> 3.0
Feature 'NumTotalTrades': 19.0 -> 18.0
Feature 'PercentInstallTrades': 16.0 -> 15.0
Feature 'MSinceMostRecentInqexcl7days': 0.0 -> -8.0
Feature 'NetFractionRevolvingBurden': 54.0 -> 55.0
Feature 'NetFractionInstallBurden': -8.0 -> -9.0
Feature 'NumRevolvingTradesWBalance': 5.0 -> 6.0
Feature 'NumInstallTradesWBalance': 1.0 -> 2.0


In [24]:
d_cf = {f"Cf{f.name}": [f.value.as_number()] for f in expl_features}

In [ ]:
import utils

cf_df = utils.save_result(
    original=X, cf=d_cf, score=cf_p, method=["TrustyAI"], model=["XGBoost"]
)

## MLP/Adam

### Load model

In [26]:
from joblib import load

mlp_model = load('../../models/mlp.joblib') 

In [27]:
mlp_predict = predict(mlp_model)

In [28]:
mlp_predict([PredictionInput(input_feature)])[0].outputs.get(0).toString()

'Output{value=0, type=number, score=0.7968092578961794, name='RiskPerformance'}'

In [29]:
mlp_model.predict_proba(X.to_numpy().reshape(1, -1))

array([[0.79680926, 0.20319074]])

In [30]:
mlp_provider = Model(mlp_predict)

In [31]:
%%time

explanation = explainer.explain(prediction, mlp_provider)

CPU times: user 5min, sys: 50.8 s, total: 5min 50s
Wall time: 59.7 s


In [32]:
expl_features = [e.asFeature() for e in explanation.entities]

In [33]:
cf_p = mlp_predict([PredictionInput(expl_features)])[0].outputs.get(0)
cf_p = {
    "GoalValue": [cf_p.getValue().asNumber()],
    "GoalName": [str(cf_p.getName())],
    "GoalScore": [cf_p.getScore()],
}
print(cf_p)

{'GoalValue': [1.0], 'GoalName': ['RiskPerformance'], 'GoalScore': [0.506854441366209]}


In [34]:
show_changes(explanation, input_feature)

Feature 'AverageMInFile': 112.0 -> 113.0
Feature 'NumTrades60Ever2DerogPubRec': 2.0 -> 1.0
Feature 'PercentTradesNeverDelq': 68.0 -> 69.0
Feature 'MaxDelq2PublicRecLast12M': 2.0 -> 17.0
Feature 'NumRevolvingTradesWBalance': 5.0 -> 6.0
Feature 'NumInstallTradesWBalance': 1.0 -> 2.0
Feature 'NumBank2NatlTradesWHighUtilization': 2.0 -> 1.0
Feature 'PercentTradesWBalance': 67.0 -> 68.0


In [35]:
d_cf = {f"Cf{f.name}": [f.value.as_number()] for f in expl_features}

In [ ]:
cf_mlp_df = utils.save_result(
    original=X, cf=d_cf, score=cf_p, method=["TrustyAI"], model=["MLP"]
)

In [39]:
final_df = pd.concat([cf_df, cf_mlp_df])
final_df.T

,0,0
ExternalRiskEstimate,63,63
MSinceOldestTradeOpen,309,309
MSinceMostRecentTradeOpen,36,36
AverageMInFile,112,112
NumSatisfactoryTrades,17,17
NumTrades60Ever2DerogPubRec,2,2
NumTrades90Ever2DerogPubRec,1,1
PercentTradesNeverDelq,68,68
MSinceMostRecentDelq,7,7
MaxDelq2PublicRecLast12M,2,2


In [40]:
final_df.to_csv("../../results/cf-trustyai.csv")